In [1]:
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import RichProgressBar
import pandas as pd

In [ ]:
from ray.train.lightning import RayDDPStrategy, RayLightningEnvironment, RayTrainReportCallback, prepare_trainer
from ray import tune
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer
from ray.tune import CLIReporter
from lightning import Trainer
from lightning.pytorch import seed_everything
import pandas as pd
import os


from utils.configurations import configs
from utils.constants import RANDOM_STATE
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier




class deepscreen_hyperparameter_tuneing:
    def __init__(self,data:pd.DataFrame,search_space:dict,target:str,max_epochs:int = 200,data_split_mode:str="non_random_split",scheduler_type = "BOHB", grace_period:int=90,metric:str="val_mcc",mode:str="max",num_workers:int=1,num_samples:int=10,experiments_result_path="../../.experiments/"):
        seed_everything(RANDOM_STATE,True)

        self.data = data
        self.search_space = search_space
        self.num_samples =  num_samples
        self.target = target
        self.data_split_mode = data_split_mode
        self.experiment_path = experiments_result_path
        self.experiment_result_path = os.path.join(experiments_result_path,self.target)
        self.max_epochs = max_epochs
        if not os.path.exists(self.experiment_result_path):
                os.makedirs(self.experiment_result_path)
        self.use_gpu = configs.get_gpu_number() > 0

        self.scheduler = HyperBandForBOHB(
                            time_attr="training_iteration",
                            max_t=self.max_epochs,
                            reduction_factor=3,
                            stop_last_trials=True,
                        )
        
        self.scaling_config = ScalingConfig(
            num_workers=1, use_gpu=self.use_gpu, resources_per_worker={"CPU": configs.get_cpu_number(), "GPU": configs.get_gpu_number()}
            )
        
        self.search_algorithm = TuneBOHB()

        self.run_config = RunConfig(
            stop={"training_iteration": self.max_epochs},
            checkpoint_config=CheckpointConfig(
                num_to_keep=2,
                checkpoint_score_attribute=metric,
                checkpoint_score_order=mode,
                
            ),
            progress_reporter=CLIReporter(max_column_length=10,max_report_frequency=15)
            )

        self.ray_trainer = TorchTrainer(
            self._train_func,
            scaling_config=self.scaling_config,
            run_config=self.run_config,
        )

    def _train_func(self,config):
        dm = DEEPscreenDataModule(
             data=self.data,
             target_id=self.target,
             batch_size=config["batch_size"],
             experiment_result_path=self.experiment_result_path,
             data_split_mode=self.data_split_mode,
             tmp_imgs=configs.get_use_tmp_imgs())
        model = DEEPScreenClassifier(**config,experiment_result_path=self.experiment_result_path)

        trainer = Trainer(
            devices="auto",
            accelerator="auto",
            strategy=RayDDPStrategy(),
            callbacks=[RayTrainReportCallback()],
            plugins=[RayLightningEnvironment()],
            deterministic=True,
            enable_progress_bar=False,
            enable_model_summary=False,
        )
        trainer = prepare_trainer(trainer)
        trainer.fit(model, datamodule=dm)

    def tune_deepscreen(self):

        tuner = tune.Tuner(
            self.ray_trainer,
            param_space={"train_loop_config": self.search_space},
            tune_config=tune.TuneConfig(
                num_samples=self.num_samples,
                scheduler=self.scheduler,
                search_alg=self.search_algorithm
            ),
        )
        return tuner.fit()

In [9]:
data = pd.read_csv("../../.data/processed/CHEMBL5567.csv")
search_space_deepscreen = {
        'fully_layer_1': tune.choice([16, 32, 128, 256, 512]),
        'fully_layer_2': tune.choice([16, 32, 128, 256, 512]),
        'learning_rate': tune.choice([0.0005, 0.0001, 0.005, 0.001, 0.01]),
        'batch_size': tune.choice([32, 64]),
        'drop_rate': tune.choice([0.5, 0.6, 0.8]),
    }
tuner = deepscreen_hyperparameter_tuneing(data,search_space_deepscreen,"chembl5567",num_workers=2)
result = tuner.tune_deepscreen()

Seed set to 123
2024-02-08 21:16:26,085	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-02-08 21:16:26 (running for 00:00:00.22)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:16:41 (running for 00:00:15.31)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:16:56 (running for 00:00:30.40)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:17:11 (running for 00:00:45.49)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/

2024-02-08 21:17:26,489	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 49.0 CPUs and 0 GPUs per trial, but the cluster only has 48.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


== Status ==
Current time: 2024-02-08 21:17:26 (running for 00:01:00.60)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:17:41 (running for 00:01:15.68)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:17:56 (running for 00:01:30.78)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:18:11 (running for 00:01:45.87)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/

2024-02-08 21:18:26,554	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 49.0 CPUs and 0 GPUs per trial, but the cluster only has 48.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


...

== Status ==
Current time: 2024-02-08 21:40:34 (running for 00:24:08.79)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:40:49 (running for 00:24:23.88)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:41:05 (running for 00:24:38.97)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:41:20 (running for 00:24:54.06)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/

2024-02-08 21:41:27,906	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 49.0 CPUs and 0 GPUs per trial, but the cluster only has 48.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


== Status ==
Current time: 2024-02-08 21:41:35 (running for 00:25:09.15)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-02-08 21:41:50 (running for 00:25:24.24)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)




2024-02-08 21:41:54,262	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-02-08 21:41:54,280	INFO tune.py:1042 -- Total run time: 1528.19 seconds (1528.16 seconds for the tuning loop).
2024-02-08 21:41:54,282	WARNING tune.py:1057 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26", trainable=...)
2024-02-08 21:41:54,296	WARNING experiment_analysis.py:193 -- Failed to fetch metrics for 10 trial(s):
- TorchTrainer_515e0_00000: FileNotFoundError('Could not fetch metrics for TorchTrainer_515e0_00000: both result.json and progress.csv were not found at /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26/TorchTrainer_515e0_00000_0_batch_size=32,drop_rate=0.6000,fully_layer_1=128,fully_layer_2=512,le

== Status ==
Current time: 2024-02-08 21:41:54 (running for 00:25:28.16)
Using MedianStoppingRule: num_stopped=0.
Logical resource usage: 0/48 CPUs, 0/0 GPUs
Result logdir: /home/sjinich/ray_results/TorchTrainer_2024-02-08_21-16-26
Number of trials: 10/10 (10 PENDING)


